# Spatial Relationships between LR Pairs and cell Types using MISTy 

### Import packages

In [ ]:
import scanpy as sc
import decoupler as dc
import plotnine as p9
import liana as li
import os

datadir = '../../datasets/Hands_on_2_LIANA_MistY/'

### Import functions needed for running MISTy 

In [ ]:
from liana.method import MistyData, genericMistyData
from liana.method.sp import RandomForestModel, LinearModel, RobustLinearModel

### Load and Normalize Data

In [ ]:
adata = sc.read(os.path.join(datadir, "kuppe_heart19.h5ad"))

In [ ]:
adata.layers['counts'] = adata.X.copy()
sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.log1p(adata)

In [ ]:
# Rename to more informative names
full_names = {'Adipo': 'Adipocytes',
              'CM': 'Cardiomyocytes',
              'Endo': 'Endothelial',
              'Fib': 'Fibroblasts',
              'PC': 'Pericytes',
              'prolif': 'Proliferating',
              'vSMCs': 'Vascular_SMCs',
              }
# but only for the ones that are in the data
adata.obsm['compositions'].columns = [full_names.get(c, c) for c in adata.obsm['compositions'].columns]

In [ ]:
##### Extract Cell type Composition
comps = li.ut.obsm_to_adata(adata, 'compositions')

In [ ]:
comps

In [ ]:
##### Compute Spatial Neighbors
li.ut.spatial_neighbors(adata, bandwidth=150, cutoff=0.1, kernel='gaussian', set_diag=True)

In [ ]:
li.pl.connectivity(adata, idx=0, figure_size=(6,5))

In [ ]:
# Compute LIANA's bivariate local spatial functions
lrdata = li.mt.bivariate(adata,
                resource_name='consensus', # uses HUMAN gene symbols
                local_name='cosine', # Name of the local function
                global_name="morans", # Name global function (or 'lee')
                n_perms=None, # Number of permutations to calculate a p-value
                mask_negatives=False, # Whether to mask LowLow/NegativeNegative interactions
                add_categories=True, # Whether to add local categories to the results
                nz_prop=0.3, # Minimum expr. proportion for ligands/receptors and their subunits
                use_raw=False,
                verbose=True
                )

In [ ]:
# or load the lrdata
lrdata = sc.read("lrdata.h5ad")

In [ ]:
lrdata

In [ ]:
# # select the top 200 lr pairs based on mean
# top_vars = lrdata.var.sort_values("mean", ascending=False).head(300).index
# lrdata_top = lrdata[:, top_vars].copy()

In [ ]:
# you can change the connectivity 
# li.ut.spatial_neighbors(lrdata, bandwidth=500, cutoff=0.1, kernel='gaussian', set_diag=True)
# li.pl.connectivity(lrdata, idx=0, figure_size=(6,5))

In [ ]:
misty = MistyData(data={"intra": comps, "lrs": lrdata})

In [ ]:
misty

## Learn Relationships with MISTy

This takes a while for, so you can instead just load the precomputed results. 
This code was used: 
```python
misty(model=LinearModel, n_jobs=-1, verbose = True, bypass_intra=True)
misty.write_h5mu('misty_mudata.h5mu'))
```

In [ ]:
import muon as mu
misty = mu.read_h5mu(os.path.join(datadir, 'misty_mudata.h5mu'))
misty

In [ ]:
misty.uns['target_metrics'].head()

In [ ]:
li.pl.target_metrics(misty, stat='multi_R2', return_fig=True)

In [ ]:
li.pl.target_metrics(misty, stat='gain_R2')

Finally, using the information above we know which variables are best explained by our model, and we know which view explains them best. 
So, we can now also see what are the specific variables that explain each target best:

In [ ]:
misty.uns['interactions'].sort_values("importances", ascending=False).head(10)

In [ ]:
(
    li.pl.interactions(misty, view='lrs', top_n=40) + 
    p9.labs(x='Ligand-Receptor Pair', y='Cell type') +
    p9.theme_bw(base_size=12) +
    p9.theme(axis_text_x=p9.element_text(rotation=90, size=10)) +
    # change to blue-red
    p9.scale_fill_gradient2(low='blue', mid='white', high='red')+
    p9.theme(figure_size=(10, 5)) 
)

In [ ]:
sc.pl.spatial(comps,  
              color=['Proliferating'], 
              cmap='RdBu_r', 
              size=1.3,
              vmin = 0.05, vmax = 0.3)


In [ ]:
sc.pl.spatial(lrdata, color=['MMRN2^CD248'],  cmap='RdBu_r', size=1.3)